In [ ]:
import gate_api
from gate_api.exceptions import ApiException, GateApiException

In [ ]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
from datetime import date
import time
import plotly
import plotly.graph_objects as go
from matplotlib import pyplot
import os

In [ ]:
configuration = gate_api.Configuration(
    host = "https://api.gateio.ws/api/v4"
)


api_client = gate_api.ApiClient(configuration)
# Create an instance of the API class
api_instance = gate_api.SpotApi(api_client)



In [ ]:
def readingInput(filename):    
    input_file = open(filename,'r')
    inputs = input_file.readline()
    inputs = int(inputs)
    if inputs == 0:
        print('number of inputs is 0')
        return
    
    channel_length = input_file.readline()
    input_file.seek(0)
    trade_pairs=[]
    x = input_file.readlines()
    #print(trades)
    #for i in range(len(x)-1):
        
        #x[i]=x[i][:-2]
    x = x[2:]
    for i in x:
        trade_pairs.append(i.split(sep=', '))
    return inputs,channel_length,trade_pairs

        

In [ ]:
inputs,channel_length,trade_pairs = readingInput('inputfile.txt')
#print(trade_pairs)

In [ ]:
def createDataFrame(trade_info):
    
    currency_pair = trade_info[0]+'_USDT' 
    
    time1 = trade_info[2].split(sep='/')
    if trade_info[3][-1]=='\n':
        time2 = trade_info[3][:-1]
    else:
        time2 = trade_info[3]
    #print(time2)
    time2= time2.split(sep='/')
    epoch = date(1970, 1, 1)
    date1 = date(int(time1[-1]),int(time1[-3]),int(time1[-2]))
    date2 = date(int(time2[-1]),int(time2[-3]),int(time2[-2]))
    diff1 = date1-epoch
    diff2 = date2-epoch
    
    ts1 =  int(diff1.total_seconds())
    ts2 = int(diff2.total_seconds())
    #print(date1,date2)
    #print(ts1,ts2)
    _from = ts1 
    to = ts2
    interval = trade_info[1]
    interval_map={'10s':10,'5m':300,'10m':600,'15m':900,'30m':1800,'1h':3600,'4h':14400,'8h':28800,'1d':86400,'7d':604800}
    
    if interval_map.get(interval)==None:
        print('Invalid interval')
        return
    if (ts2-ts1)/interval_map[interval] > 1000 :
        print(ts2-ts1/interval_map[interval])
        print('number of points exceeded')
        return
    if (ts2-ts1)/interval_map[interval] < 1 :
        print('number of points less than 1')
        return
    
    
    try:
        # Market candlesticks
        api_response = api_instance.list_candlesticks(currency_pair, _from=_from , to = to, interval=interval)

        #print(api_response)
    except GateApiException as ex:
        print("Gate api exception, label: %s, message: %s\n" % (ex.label, ex.message))
    except ApiException as e:
        print("Exception when calling SpotApi->list_candlesticks: %s\n" % e)
    for i in api_response:
        
        i = i.remove(i[6])
    #print(api_response)
    df = pd.DataFrame(api_response, columns=['unix timestamp','trading volume','close','high','low','open'])
    
    df['unix timestamp'] = df['unix timestamp'].astype('int')
    df['trading volume'] = df['trading volume'].astype('float64')
    df['close'] = df['close'].astype('float64')
    df['open'] = df['open'].astype('float64')
    df['low'] = df['low'].astype('float64')
    df['high'] = df['high'].astype('float64')
    return df

In [ ]:
def highs_above(slope, intercept, df, window, start):
    count = 0
    inline = 0
    for i in range(start,start+window):
        if df['high'].iloc[i] > slope*i + intercept:
            count+=1
        elif df['high'].iloc[i] == slope*i + intercept:
            inline+=1
    return count,inline
    

In [ ]:
def lows_below(slope, intercept, df, window, start):
    count = 0
    inline=0
    for i in range(start,start+window):
        if df['low'].iloc[i] < slope*i + intercept:
            count+=1
        elif df['low'].iloc[i] == slope*i + intercept:
            inline+=1
    return count,inline

In [ ]:
def make_channel(df,window,start,step):
    
    #arrays for max and min of wicks
    maxwick = np.array([])
    minwick = np.array([])
    idminwick = np.array([])
    idmaxwick = np.array([])
    
    #arrays for extreme points of window
    maxmin = np.array([])
    ids = np.array([])
    
    #arrays for max and min of candles
    maxcandle=np.array([])
    idmaxcandle=np.array([])
    mincandle=np.array([])
    idmincandle=np.array([])
    
    
    for i in range(start,start+window,step):
        minwick = np.append(minwick, df.low.iloc[i:i+step].min())        
        idminwick = np.append(idminwick, df.low.iloc[i:i+step].idxmin())
        maxwick = np.append(maxwick, df.high.iloc[i:i+step].max())    
        idmaxwick = np.append(idmaxwick, df.high.iloc[i:i+step].idxmax())
        
        maxcandle = np.append(maxcandle, max(df.open.iloc[i:i+step].max(),df.close.iloc[i:i+step].max()))
        if df.open.iloc[i:i+step].max()>df.close.iloc[i:i+step].max():
            idmaxcandle = np.append(idmaxcandle, df.open.iloc[i:i+step].idxmax())
        else:
            idmaxcandle = np.append(idmaxcandle, df.close.iloc[i:i+step].idxmax())
            
         
        mincandle = np.append(mincandle, min(df.open.iloc[i:i+step].min(),df.close.iloc[i:i+step].min()))
        
        if df.open.iloc[i:i+step].min()<df.close.iloc[i:i+step].min():
            idmincandle = np.append(idmincandle, df.open.iloc[i:i+step].idxmin())
        else:
            idmincandle = np.append(idmincandle, df.close.iloc[i:i+step].idxmin())
        
        maxmin = np.append(maxmin, df.low.iloc[i:i+step].min())
        ids = np.append(ids, df.low.iloc[i:i+step].idxmin())
        maxmin = np.append(maxmin, df.high.iloc[i:i+step].max())
        ids = np.append(ids, df.high.iloc[i:i+step].idxmax())
        
    slope, intercept = np.polyfit(ids,maxmin,1)
    intercept_plus_top = (df.high.iloc[idmaxwick] - slope*idmaxwick).max()
    intercept_plus_buttom = (df.high.iloc[idmaxcandle] - slope*idmaxcandle).max()
    
    intercept_minus_buttom = (df.low.iloc[idminwick] - slope*idminwick).min()   
    intercept_minus_top = (df.low.iloc[idmincandle] - slope*idmincandle).min()
    
    plus_diff = intercept_plus_top - intercept_plus_buttom
    minus_diff = intercept_minus_top - intercept_minus_buttom
    
    dec = plus_diff/100
    
    inc = minus_diff/100
    
    for i in range(101):
        intercept_plus_opt = intercept_plus_top - i*dec 
        x,y = highs_above(slope,intercept_plus_opt,df,window,start)
        
        if x>3:
            break
            
    for i in range(101):
        intercept_minus_opt = intercept_minus_buttom + i*inc 
        x,y = lows_below(slope,intercept_minus_opt,df,window,start)
        
        if x>3:
            break
    
        
    
    if intercept_plus_opt-intercept< 0.1* intercept and intercept - intercept_minus_opt< 0.1* intercept:
        dfpl = df[start:start+window]
        fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])
        fig.add_trace(go.Scatter(x=idminwick, y=slope*idminwick + intercept_minus_opt , mode='lines', name='min'))
        fig.add_trace(go.Scatter(x=idmaxwick, y=slope*idmaxwick + intercept_plus_opt, mode='lines', name='max'))
        fig.show()
        
        if not os.path.exists("plots"):
            os.mkdir("plots")
        name  = 'plot' + str(start)    
        fig.write_image("plots/" + name + ".png")
        return True
    return False

In [ ]:
min_channel_length = int(channel_length)
step  = 5

for trade_pair in trade_pairs:
    data = createDataFrame(trade_pair)
    name = trade_pair[0]
    channels = 0
    for index in range(0,len(data.index)-100,step):

        for length in range(100,min_channel_length,-step):    
            plot_funct = make_channel(data, length, index, step)
            if plot_funct==True:
                channels+=1

                break

    for index in range(len(data.index)-100,len(data.index)-min_channel_length,step):

        for length in range(len(data.index)-index,min_channel_length,-step):    
            plot_funct = make_channel(data, length, index, step)
            if plot_funct==True:
                channels+=1
                break
                
    print(name + ':' + str(channels))
    
        
            
            
        
        
    



